In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature = 0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator = "\n",
    chunk_size = 600,
    chunk_overlap = 100,
)

loader = UnstructuredFileLoader("./files/1984_chap_one.docx")

doc = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(doc, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer jus say you don't know, don't make it up:\n\n{context}"),
    ("human", "{question}")
])

chain = {"context": retriver, "question": RunnablePassthrough()} | prompt | llm

chain.invoke("Describe Victory Mansions")


AIMessage(content='Victory Mansions is a building where Winston Smith resides. It is a run-down apartment complex with glass doors, gritty dust, and a hallway that smells of boiled cabbage and old rag mats. The flat is seven flights up, and the building has a faulty lift due to the electricity being cut off during daylight hours as part of an economy drive. Inside the flat, there is a telescreen that cannot be completely shut off, and a poster with an enormous face that reads "BIG BROTHER IS WATCHING YOU."', response_metadata={'token_usage': <OpenAIObject at 0x1480bf8f0> JSON: {
  "prompt_tokens": 1956,
  "completion_tokens": 106,
  "total_tokens": 2062
}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4690918e-c39e-4267-af8c-ad9646acf1b4-0')